In [1]:
import os
import sys
import pickle
import numpy as np
from numpy import gradient
from math import pi, ceil
import matplotlib.pyplot as plt
from packages import data_container
from packages.data_container import Data
from packages.helper import traj_speed, beta, min_dist, d_psi, min_sep
from collections import defaultdict
# For pickle to load the Data object, which is defined in packages.data_container
sys.modules['data_container'] = data_container

In [2]:
'''Import data from csv to Data object'''
header=('subj_x', 'subj_y', 'subj_z', 'subj_yaw', 'subj_pitch', 'subj_row')
info={'p_goal':[], 'subj_id':[], 'trial_id':[], 'stimuli_onset':[],  'leader_s0':[], 'goal_d0':[], 'goal_angle':[],
      'stimuli_out':[], 'w_goal': 0.4, 'ps_trial':[], 'ps_subj':[], 'time_stamp':[],
      'p_subj':[], 'v_subj':[], 'a_subj':[]}
data = Data(Hz=90, header=header, info=info)
info['experiment_name'] = 'Bai_movObst1b'
folder = os.path.join(os.getcwd(), os.pardir, 'Raw_Data', 'Bai_movObst1b')
os.listdir(folder)

for file in os.listdir(folder):
    # Import output
    file_path = os.path.abspath(os.path.join(folder, file))
    subj_id = int(file[-6:-4])
    with open(file_path, 'r') as f:
        trials = [line.split(',') for line in f.read().split('\n')]
    head = 0
    while head + 1 < len(trials):
        for i in range(head+1, len(trials)):
            if trials[i][0] == '':
                tail = i - 1
                break
        trial_id = int(trials[head+1][1])
        time_stamp = [float(line[0]) for line in trials[head+3:tail+1]]
        s0 = float(trials[head+1][3])
        d0 = float(trials[head+1][5])
        angle = float(trials[head+1][7])
        subj = np.array([[float(val) for val in line[1:7]] for line in trials[head+3:tail+1]])
        p0 = Data.filter(subj[:,[0,2]], time_stamp, 90, 4, 0.6)
        v0 = gradient(p0, axis=0) * data.Hz
        a0 = gradient(v0, axis=0) * data.Hz
        goal = np.array([[float(line[-3]), float(line[-2])] for line in trials[head+3:tail+1]])
        stimuli_onset = stimuli_out = None
        if s0:
            for i in range(head+3, tail+1):
                if trials[i][-1] == 'True' or trials[i][-1] == 'TRUE':
                    stimuli_onset = i - head - 3
                    break
            stimuli_out = len(time_stamp) - 1
        data.add_traj(subj)
        ps_trial = np.mean(traj_speed(subj[-2*data.Hz:-data.Hz,[0,2]], data.Hz))            
        data.add_info({'p_goal':goal, 'subj_id':subj_id, 'trial_id':trial_id,
                       'stimuli_onset':stimuli_onset, 'stimuli_out':stimuli_out, 'time_stamp':time_stamp, 
                       'ps_trial':ps_trial, 'ps_subj':0, 'leader_s0':s0, 'goal_d0':d0, 'goal_angle':angle,
                       'p_subj':p0, 'v_subj':v0, 'a_subj':a0})
        head = tail + 1

'''Dump bottom 1% trials that have the smallest minimal speed'''
vs = []
for i in range(len(data.trajs)):
    if i in data.dump or data.info['leader_s0'][i] == 0:
        continue
    t0 = data.info['stimuli_onset'][i]
    t1 = data.info['stimuli_out'][i]
    if t1 - t0 > 1000:
        data.dump[i] = 'Trial took more than 1000 frames'
    s = min(traj_speed(data.info['p_subj'][i][t0:t1], Hz=data.Hz))
    vs.append((s, i))
vs.sort()
n = int(len(vs) / 100) # 1% of trials
print(f'The cutoff speed is {vs[n]}')
for i in [x[1] for x in vs[:n]]:
    data.dump[i] = 'min speed below bottom 1%'
        
ps_subj = defaultdict(int)
for i in range(len(data.trajs)):
    if data.info['leader_s0'][i] == 0:
        subj_id = data.info['subj_id'][i]
        ps_subj[subj_id] += data.info['ps_trial'][i] / 10  # Because there were 10 freewalk trials
for i in range(len(data.trajs)):
    subj_id = data.info['subj_id'][i]
    data.info['ps_subj'][i] = ps_subj[subj_id]
outfile = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Raw_Data', 'Bai_movObst1b_data.pickle'))       
# with open(outfile, 'wb') as f:   
#     pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)  

(0.4307497923198857, 307)


In [3]:
'''Save a 30 Hz version for fitting'''
# Convert to 30 Hz for fitting
data.Hz = 30
for i in range(len(data.trajs)):
    data.trajs[i] = data.trajs[i][::3]
    data.info['p_goal'][i] = data.info['p_goal'][i][::3]
    data.info['time_stamp'][i] = data.info['time_stamp'][i][::3]
    data.info['p_subj'][i] = data.info['p_subj'][i][::3]
    data.info['v_subj'][i] = data.info['v_subj'][i][::3]
    data.info['a_subj'][i] = data.info['a_subj'][i][::3]
    if data.info['stimuli_onset'][i]:
        data.info['stimuli_onset'][i] = ceil(data.info['stimuli_onset'][i] / 3)
        data.info['stimuli_out'][i] = data.info['stimuli_out'][i] // 3
outfile = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Raw_Data', 'Bai_movObst1b_data_30Hz.pickle'))       
with open(outfile, 'wb') as f:   
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

In [16]:
set(data.info['subj_id'])

{0, 1, 2, 3, 4, 6, 7, 9, 10, 11, 12, 13}

In [38]:
set(data.info['leader_s0'])

{0.0, 0.5, 1.4}

In [69]:
length = []
for i in range(len(data.trajs)):    
    if data.info['stimuli_out'][i] and i not in data.dump:
#         print(i,  data.info['stimuli_onset'][i], data.info['stimuli_out'][i])
        length.append((data.info['stimuli_out'][i] - data.info['stimuli_onset'][i], i, data.info['subj_id'][i], data.info['trial_id'][i]))
length.sort()
length[-50:]
# print(len(length), len(data.trajs))

[(654, 1356, 12, 57),
 (657, 1111, 10, 72),
 (657, 1407, 12, 108),
 (658, 952, 9, 43),
 (659, 1093, 10, 54),
 (660, 1466, 13, 37),
 (661, 1079, 10, 40),
 (661, 1128, 10, 89),
 (661, 1405, 12, 106),
 (663, 1290, 11, 121),
 (664, 1470, 13, 41),
 (667, 1138, 10, 99),
 (669, 953, 9, 44),
 (670, 1391, 12, 92),
 (673, 949, 9, 40),
 (674, 1119, 10, 80),
 (674, 1347, 12, 48),
 (675, 1493, 13, 64),
 (678, 1485, 13, 56),
 (679, 1045, 10, 6),
 (679, 1109, 10, 70),
 (679, 1517, 13, 88),
 (682, 1211, 11, 42),
 (683, 1047, 10, 8),
 (683, 1127, 10, 88),
 (686, 944, 9, 35),
 (688, 1146, 10, 107),
 (688, 1164, 10, 125),
 (689, 1140, 10, 101),
 (690, 396, 3, 7),
 (690, 921, 9, 12),
 (692, 1481, 13, 52),
 (694, 1132, 10, 93),
 (695, 1137, 10, 98),
 (697, 1121, 10, 82),
 (697, 1153, 10, 114),
 (699, 1160, 10, 121),
 (700, 1044, 10, 5),
 (703, 1437, 13, 8),
 (707, 1439, 13, 10),
 (719, 1048, 10, 9),
 (719, 1154, 10, 115),
 (720, 1062, 10, 23),
 (725, 1066, 10, 27),
 (729, 1049, 10, 10),
 (736, 1124, 10, 85

In [67]:
'''check redundant trials'''
trials = defaultdict(int)
for i in range(len(data.trajs)):
    subj_id = data.info['subj_id'][i]
    trial_id = data.info['trial_id'][i]
    trials[(subj_id, trial_id)] += 1
for k, v in trials.items():
    if v > 1:
        print(k, v)

In [ ]:
i = 1
print(data.info['stimuli_onset'][i])
print(data.info['time_stamp'][i][297])
print(data.info['stimuli_out'][i], len(data.info['time_stamp'][i]))
print(data.info['leader_s0'][i], data.info['goal_d0'][i], data.info['goal_angle'][i])
print(data.info['ps_trial'][i], data.info['ps_subj'][i])
print(data.info['p_subj'][i])
print(data.info['v_subj'][i][-180:-90])
print(data.info['a_subj'][i])

In [75]:
# Check imported data
vals = []
%matplotlib qt
for i in range(len(data.trajs)):
    if i in data.dump:
        continue
    val = data.info['ps_trial'][i]
    vals.append(val)
    p_goal = np.array(data.info['p_goal'][i])
    p_subj = data.info['p_subj'][i]
    onset = data.info['stimuli_onset'][i]
    i_subj = data.info['subj_id'][i]
    angle = data.info['goal_angle'][i]
#     plt.plot(vals)
    plt.plot(data.info['p_subj'][i][:,0], data.info['p_subj'][i][:,1], data.Hz)
#     if i_subj == 0:
#         plt.plot(traj_speed(data.info['p_subj'][i], data.Hz))
#     if angle == -20:
#         plt.plot(data.info['p_subj'][i][:,0], data.info['p_subj'][i][:,1])